# 依不同環境安裝套件並掛載資料夾

In [1]:
import os, site, sys
from pathlib import Path

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')
  !pip install torch --quiet
  !pip install transformers datasets evaluate peft accelerate sentencepiece --quiet
  !pip install numpy matplotlib tqdm --quiet
  !pip install islab-opendeid --quiet
else:
  %pip install torch --quiet
  %pip install transformers datasets evaluate peft accelerate sentencepiece --quiet
  %pip install numpy matplotlib tqdm --quiet
  %pip install islab-opendeid --quiet
  %pip install ipywidgets nbformat nbclient widgetsnbextension pandas-profiling --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
Mounted at /content/drive


# 重載套件並切換工作目錄

In [2]:
import os, site, sys
from pathlib import Path

if "google.colab" in sys.modules:
  OUT_DIR_PATH = Path("/content/drive/MyDrive/aicup/out")
  RES_DIR_PATH = Path("/content/drive/MyDrive/aicup/res")
  SRC_DIR_PATH = Path("/content/drive/MyDrive/aicup/src")
else:
  OUT_DIR_PATH = Path(Path.cwd().parent, "out")
  RES_DIR_PATH = Path(Path.cwd().parent, "res")
  SRC_DIR_PATH = Path(Path.cwd().parent, "src")

site.main()
os.chdir(SRC_DIR_PATH)

# 定義 TsvDatasetHelper 以便後續讀取
https://huggingface.co/docs/datasets/main/en/loading#csv

In [3]:
from pathlib import Path
from datasets import load_dataset, Features, Value

class TsvDatasetHelper:
  TsvFeatures = Features(
    {
      "fid": Value("string"),
      "idx": Value("int64"),
      "content": Value("string"),
      "label": Value("string"),
    }
  )
  @staticmethod
  def get_dataset(tsv_path: Path):
    data_files = tsv_path.as_posix()
    print(data_files)
    return load_dataset(
      path="csv",
      delimiter="\t",
      data_files=data_files,
      features=TsvDatasetHelper.TsvFeatures,
      column_names=TsvDatasetHelper.TsvFeatures.keys(),
      keep_default_na=False,
    )


# 載入 TSV Dataset

In [4]:
tsv_path = Path(RES_DIR_PATH, "Demo_Dataset/opendid_set1.tsv")
dataset = TsvDatasetHelper.get_dataset(tsv_path)
dataset

/content/drive/MyDrive/aicup/res/Demo_Dataset/opendid_set1.tsv


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['fid', 'idx', 'content', 'label'],
        num_rows: 85736
    })
})

# 印出檢視 TSV Dataset

In [7]:
from pprint import pprint

pprint(dataset['train'][0])
pprint(dataset['train'][1])
pprint(dataset['train'][7])

/content/drive/MyDrive/aicup/res/Demo_Dataset/opendid_set1.tsv
DatasetDict({
    train: Dataset({
        features: ['fid', 'idx', 'content', 'label'],
        num_rows: 85736
    })
})
{'content': 'Episode No:  09F016547J',
 'fid': '10',
 'idx': 1,
 'label': 'IDNUM: 09F016547J'}
{'content': '091016.NMT',
 'fid': '10',
 'idx': 25,
 'label': 'MEDICALRECORD: 091016.NMT'}
{'content': 'D.O.B:  24/8/1993',
 'fid': '10',
 'idx': 114,
 'label': 'DATE: 24/8/1993=>1993-08-24'}


# 隨機挑選 20000 個 Sample 來做 Demo

In [15]:
from torch.utils.data import random_split

print(len(dataset['train']))
sub_datasets = random_split(dataset['train'], [20000, 65736])
print(len(sub_datasets[0]))

for i in range(4):
  pprint(sub_datasets[0][i])

85736
20000
{'content': ['Left ovary stromal invasion (0.5 mm)',
             'Extensive adenomyosis is present.',
             'The right ovary contains a luteinised follicular cyst, but shows '
             'no evidence of tumour.',
             'Sections of the ovaries and fallopian tubes are unremarkable.'],
 'fid': ['file4704', 'file1163', 'file5048', '97'],
 'idx': [6005, 4690, 8209, 4595],
 'label': ['PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL']}


# 定義 TokenHelper 以便後續處理

In [16]:
class TokenHelper:
  Bos = "<|endoftext|>"
  Eos = "<|END|>"
  Pad = "<|pad|>"
  Sep = "\n\n####\n\n"
  SpecialTokens = {
    "bos_token": Bos,
    "eos_token": Eos,
    "pad_token": Pad,
    "sep_token": Sep,
  }

  @staticmethod
  def add_token(med_info, phi_info):
    return "{}{}{}{}{}".format(
      TokenHelper.Bos,
      med_info,
      TokenHelper.Sep,
      phi_info,
      TokenHelper.Eos,
    )

# Data loader

In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM

plm = "EleutherAI/pythia-70m" #"EleutherAI/pythia-70m-deduped"
tokenizer = AutoTokenizer.from_pretrained(plm, revision="step3000")
tokenizer.padding_side = 'left'
num_added_toks = tokenizer.add_special_tokens(TokenHelper.SpecialTokens)
print(num_added_toks)
print(f"{tokenizer.pad_token}: {tokenizer.pad_token_id}")

3
<|pad|>: 50278


In [18]:
from torch.utils.data import DataLoader
from islab.aicup import collate_batch_with_prompt_template

train_data = list(sub_datasets[0])
train_dataloader = DataLoader(
  train_data,
  batch_size=3,
  shuffle=False,
  collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer),
)
titer = iter(train_dataloader)
tks, labels, masks= next(titer)
print(tks.shape)
next(iter(titer))

torch.Size([3, 26])


(tensor([[    0, 31965,   273,   253, 16119,   447,   285,  2965, 40113, 17080,
            403,   440, 39808,   494,    15, 50279,  6663,    42,    27,  5812,
            209, 50277],
         [50278, 50278,     0,  9601,  4058,    19,  3491,  1180,   591,   894,
             27,   337,    15,    25,    15, 50279,  6663,    42,    27,  5812,
            209, 50277],
         [50278,     0,   428, 50276,   688,  1592,  1528,    27, 50276,    23,
           5823,   313, 11144,  7288,   481, 50279,  6663,    42,    27,  5812,
            209, 50277]]),
 tensor([[    0, 31965,   273,   253, 16119,   447,   285,  2965, 40113, 17080,
            403,   440, 39808,   494,    15, 50279,  6663,    42,    27,  5812,
            209, 50277],
         [ -100,  -100,     0,  9601,  4058,    19,  3491,  1180,   591,   894,
             27,   337,    15,    25,    15, 50279,  6663,    42,    27,  5812,
            209, 50277],
         [ -100,     0,   428, 50276,   688,  1592,  1528,    27, 50276, 

In [19]:
results = tokenizer(["Lab No: 14H02780", "“STOCKDALE” 653 MONAGHAN RD"], padding=True)
print(results['input_ids'])
print()
print(results['input_ids'][0])
print(tokenizer.decode(results['input_ids'][0]))
print(results['input_ids'][1])
print(tokenizer.decode(results['input_ids'][1]))

[[50278, 50278, 50278, 50278, 50278, 50278, 21663, 1621, 27, 1638, 41, 16604, 1438], [1628, 1267, 9466, 37, 23502, 668, 721, 3357, 33995, 2696, 41, 1539, 28613]]

[50278, 50278, 50278, 50278, 50278, 50278, 21663, 1621, 27, 1638, 41, 16604, 1438]
<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|>Lab No: 14H02780
[1628, 1267, 9466, 37, 23502, 668, 721, 3357, 33995, 2696, 41, 1539, 28613]
“STOCKDALE” 653 MONAGHAN RD


In [24]:
results = tokenizer(
  [
    TokenHelper.add_token(" 9364819.RAN\\nMINTANIA, JEFFRY ", " ID: 9364819.RAN\\nNAME: MINTANIA, JEFFRY "),
    TokenHelper.add_token(" This is a sentence ", " PHI: NULL "),
  ],
  padding=True
)
print(results['attention_mask'][0])
print(tokenizer.decode(results['input_ids'][0]))
print(results['attention_mask'][1])
print(tokenizer.decode(results['input_ids'][1]))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<|endoftext|> 9364819.RAN\nMINTANIA, JEFFRY 

####

 ID: 9364819.RAN\nNAME: MINTANIA, JEFFRY <|END|>
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|endoftext|> This is a sentence 

####

 PHI: NULL <|END|>


In [ ]:
from islab.aicup import OpenDeidBatchSampler

BATCH_SIZE = 8
bucket_train_dataloader = DataLoader(
  train_data,
  batch_sampler=OpenDeidBatchSampler(train_data, BATCH_SIZE),
  collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer),
  pin_memory=True,
)

# for idx, batch in enumerate(bucket_train_dataloader):
#     print(batch)
#     print(batch[0].shape)
#     print(batch[1].shape)
#     break

# Model

In [ ]:
from peft import LoraConfig, TaskType
from peft import AutoPeftModelForCausalLM
from peft import get_peft_model
from transformers import AutoConfig

plm = "EleutherAI/pythia-70m" #"EleutherAI/pythia-70m-deduped"
peft_config = LoraConfig(
  task_type=TaskType.CAUSAL_LM,
  inference_mode=False,
  r=8, lora_alpha=32, lora_dropout=0.1,
)
# the model config to which we add the special tokens
config = AutoConfig.from_pretrained(
  plm,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id,
  pad_token_id=tokenizer.pad_token_id,
  sep_token_id=tokenizer.sep_token_id,
  output_hidden_states=False,
)

model = AutoModelForCausalLM.from_pretrained(plm, revision="step3000", config=config)
model = AutoPeftModelForCausalLM.from_pretrained(plm, revision="step3000", config=config)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

EPOCHS = 3 # CHANGE TO THE NUMBER OF EPOCHS YOU WANT
optimizer = AdamW(model.parameters(),lr=3e-5) # YOU CAN ADJUST LEARNING RATE

device = torch.device(
    "mps" if torch.backends.mps.is_available() else
    "cuda" if torch.cuda.is_available() else
    "cpu"
)

model.resize_token_embeddings(len(tokenizer))
model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [ ]:
from tqdm.auto import tqdm

global_step = EPOCHS * len(bucket_train_dataloader)
progress_bar = tqdm(range(global_step), desc = 'Step')

total_loss = 0
model.train()
for epoch in range(EPOCHS):
  model.train()
  total_loss = 0
  predictions , true_labels = [], []

  for seqs, labels, masks in bucket_train_dataloader:
    seqs = seqs.to(device)
    labels = labels.to(device)
    masks = masks.to(device)
    model.zero_grad()
    outputs = model(seqs, labels=labels, attention_mask=masks)
    logits = outputs.logits
    loss = outputs.loss
    loss = loss.mean()

    total_loss += loss.item()
    loss.backward()
    optimizer.step()
    progress_bar.update(1)
  avg_train_loss = total_loss / len(bucket_train_dataloader)
  print(f"Epoch {epoch}:\nAverage train loss: {avg_train_loss}")

Step:   0%|          | 0/7500 [00:00<?, ?it/s]

Epoch 0: Average train loss: 1.8528453432321548
Epoch 1: Average train loss: 1.4213590891242027
Epoch 2: Average train loss: 1.2397726550936699


In [ ]:
valid_data = DatasetHelper.get_dataset(
  Path(RES_DIR_PATH, "Demo_Dataset/opendid_valid.tsv")
)
valid_list = list(valid_data['train'])
valid_list[0]

{'fid': '1001', 'idx': 0, 'content': 'Episode No:  88Y206206L', 'label': ''}

In [ ]:
from tqdm.notebook import tqdm
from islab.aicup import aicup_predict
from datetime import datetime

BATCH_SIZE = 32

# datetime object containing current date and time in format YY-mm-dd_H-M-S
now_dt_string = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
path = Path(OUT_DIR_PATH, f"{now_dt_string}_answer.txt")
merged_model = model.merge_and_unload()
with open(path,'w',encoding='utf8') as f:
  for i in tqdm(range(0, len(valid_list), BATCH_SIZE)):
    with torch.no_grad():
      seeds = valid_list[i:i+BATCH_SIZE]
      outputs = aicup_predict(merged_model, tokenizer, input=seeds)
      for o in outputs:
        f.write(o)
        f.write('\n')

  0%|          | 0/1387 [00:00<?, ?it/s]